In [1]:
import numpy as np
from scipy.constants import hbar,h,e,c
from scipy.constants import elementary_charge as elc
F0 = 2.0678e-15

In [2]:
e_eff=(11.4+1)/2## for all calculations
Mhz=(2*np.pi*1e6)## divide on it to obtain value in mhz
j=0.5e-3      # critical current density nA/nm^2

In [3]:
import pandas as pd

# Q1 related parameters

In [4]:
def cap_func(Q1,Qg1,Qg2):
    return Q1+(1/Qg1+1/Qg2)**(-1)

In [5]:
def Ec(C):#fF
    return np.round((elc)**2/(2*C)/h/1e9*1e15,1) # return in MHz

In [6]:
path=r'E:\Github2.0\QCreator\designs\ThreeQubitsPlusTwoQubits\maxwell_simulations'
path=r'C:\Python\Github\QCreator\designs\ThreeQubitsPlusTwoQubits\maxwell_simulations'

In [7]:
data=pd.read_csv(path+'\Q1fine_sim.csv',sep=';')# matrix should be done on pF

In [8]:
data

,Unnamed: 0,Charge,ground,Q1bottom,Q1top,resonator
0,Charge,0.006063,-0.005913,-0.000032,-0.000117,-0.000002
1,ground,-0.005913,0.199550,-0.083040,-0.090835,-0.019762
2,Q1bottom,-0.000032,-0.083040,0.129420,-0.039168,-0.007184
3,Q1top,-0.000117,-0.090835,-0.039168,0.130870,-0.000747
4,resonator,-0.000002,-0.019762,-0.007184,-0.000747,0.027694


In [13]:
data['Q1top']

0   -0.000117
1   -0.090835
2   -0.039168
3    0.130870
4   -0.000747
Name: Q1top, dtype: float64

In [9]:
q_name='Q1'
Q1_cap_ground_top=-data[q_name+'top'][1]
Q1_cap_ground_bottom=-data[q_name+'bottom'][1]
Q1_cap_top_bottom=-data[q_name+'bottom'][3]
Q1_cap=cap_func(Q1_cap_top_bottom,Q1_cap_ground_top,Q1_cap_ground_bottom)

In [12]:
Q1_cap_top_bottom,Q1_cap_ground_bottom,Q1_cap_ground_top

(0.039168, 0.08304, 0.090835)

In [11]:
Q1_cap

0.08254938547807333

In [12]:
Ec(Q1_cap)

234.7

### Resonator coupling

In [66]:
def line_qubit_coupling(Q1g,Q2g,Q1gc,Q2gc):
    return np.abs((Q1g*Q2gc+Q2g*Q1gc)/(Q1g+Q2gc+Q2g+Q1gc))

In [70]:
def cap_pretty_print(args,coeff):
    for i in args:
        print(np.round(i*coeff,1))# return in fF

In [15]:
Q1_cap_res_top=-data[q_name+'top'][4]
Q1_cap_res_bottom=-data[q_name+'bottom'][4]
Ccr=line_qubit_coupling(Q1_cap_ground_top,Q1_cap_ground_bottom,Q1_cap_res_top,Q1_cap_res_bottom)

In [16]:
cap_pretty_print([Q1_cap_res_top,Q1_cap_res_bottom,Ccr],1e3)

0.7
7.2
3.9


In [39]:
def qubit_res_coupling(Cc,Cq,wq,wr,Zr):# capacitances in fF, frequencies in GHz
    g= Cc*10**(-12)/2*np.sqrt((2*np.pi*wq*10**(9)*Zr/(Cq*10**(-12))))*wr*10**(9)
    return np.round(g/1e6,1)# return g/2pi in MHz

In [41]:
Zr=50
omega_r=5.5
omega_q=4.5
gqr=qubit_res_coupling(Ccr,Q1_cap,omega_q,omega_r,Zr)
gqr

44.7

### Dispersive shift

In [48]:
def dispersive_shift(gqr,wq,wr,Ec): # gqr in MHz,wq,wr in GHZ, EC in MHz and positive
    chi=gqr**2/((wq-wr)*10**(3))/(1+(wq-wr)*10**(3)/(-Ec))
    return chi # in MHz

In [185]:
chi=dispersive_shift(gqr,omega_q,omega_r,Ec(Q1_cap))
chi

-0.36366055214723925

### Readout resonator properties

In [91]:
Q=10000
k=2*np.pi*omega_r*1e9/Q 
np.round(1/k*1e9,1), np.round(k/Mhz,2)# first value in ns

(289.4, 0.55)

In [99]:
Tp=(1/k)*((omega_q-omega_r)*10**3/gqr)**2
Tp*1e6#us

144.82461941326272

### Charge line

In [72]:
Q1_cap_charge_line_top=-data[q_name+'top'][0]
Q1_cap_charge_line_bottom=-data[q_name+'bottom'][0]
Cc_charge=line_qubit_coupling(Q1_cap_ground_top,Q1_cap_ground_bottom,Q1_cap_charge_line_top,Q1_cap_charge_line_bottom)
cap_pretty_print([Q1_cap_charge_line_top,Q1_cap_charge_line_bottom,Cc_charge],1e6)#in aF

116.5
31.6
72.1


In [100]:
Z0=50
Tcharge=(Cc_charge+Q1_cap)/(Z0*(2*np.pi*omega_q*Cc_charge)**2)*1e-6

In [101]:
Tcharge*1e6#in us

397.6797739128156

### Total relaxation= resonator+charge line

In [103]:
Tp*Tcharge/(Tp+Tcharge)*1e6

106.16286727591249

### Coupling between qubits

In [107]:
data=pd.read_csv(path+'\wholechip.csv',sep=',')# matrix should be done on pF

In [108]:
data

,Unnamed: 0,ground,Q1_bottom,Q1_top,Q2_bottom,Q2_top,Q3_bottom,Q3_top,resonator,T2_large,T2_small,T3_large,T3_small
0,ground,0.938199,-0.070338,-0.073785,-0.081621,-0.073800,-0.083599,-0.073120,-0.014999,-0.092547,-0.055815,-0.266416,-0.052157
1,Q1_bottom,-0.070338,0.136435,-0.038838,-0.000233,-0.000245,-0.000452,-0.000776,-0.011749,-0.002024,-0.000770,-0.010790,-0.000220
2,Q1_top,-0.073785,-0.038838,0.134090,-0.000236,-0.000330,-0.000227,-0.000380,-0.000725,-0.016342,-0.001642,-0.001472,-0.000112
3,Q2_bottom,-0.081621,-0.000233,-0.000236,0.136008,-0.038912,-0.000682,-0.000484,-0.000024,-0.001997,-0.000724,-0.010623,-0.000474
4,Q2_top,-0.073800,-0.000245,-0.000330,-0.038912,0.133401,-0.000373,-0.000264,-0.000022,-0.016196,-0.001532,-0.001487,-0.000241
5,Q3_bottom,-0.083599,-0.000452,-0.000227,-0.000682,-0.000373,0.128859,-0.039330,-0.000070,-0.000129,-0.000060,-0.002497,-0.001441
6,Q3_top,-0.073120,-0.000776,-0.000380,-0.000484,-0.000264,-0.039330,0.132443,-0.000122,-0.000152,-0.000070,-0.014997,-0.002747
7,resonator,-0.014999,-0.011749,-0.000725,-0.000024,-0.000022,-0.000070,-0.000122,0.028467,-0.000118,-0.000051,-0.000553,-0.000033
8,T2_large,-0.092547,-0.002024,-0.016342,-0.001997,-0.016196,-0.000129,-0.000152,-0.000118,0.161083,-0.030309,-0.001165,-0.000105
9,T2_small,-0.055815,-0.000770,-0.001642,-0.000724,-0.001532,-0.000060,-0.000070,-0.000051,-0.030309,0.091517,-0.000498,-0.000047


In [118]:
q_name='Q2_'
Q2_cap_ground_top=data[q_name+'top'][4]+data[q_name+'top'][3]
Q2_cap_ground_bottom=data[q_name+'bottom'][3]+data[q_name+'bottom'][4]
Q2_cap_top_bottom=-data[q_name+'bottom'][4]
Q2_cap=cap_func(Q2_cap_top_bottom,Q2_cap_ground_top,Q2_cap_ground_bottom)

In [124]:
cap_pretty_print([Q2_cap_ground_top,Q2_cap_ground_bottom,Q2_cap_top_bottom],1e3)

94.5
97.1
38.9


In [119]:
Ec(Q2_cap)

223.2

In [126]:
q_name='Q1_'
Q1_cap_ground_top=data[q_name+'top'][2]+data[q_name+'top'][1]
Q1_cap_ground_bottom=data[q_name+'bottom'][1]+data[q_name+'bottom'][2]
Q1_cap_top_bottom=-data[q_name+'bottom'][2]
Q1_cap=cap_func(Q1_cap_top_bottom,Q1_cap_ground_top,Q1_cap_ground_bottom)

In [127]:
cap_pretty_print([Q1_cap_ground_top,Q1_cap_ground_bottom,Q1_cap_top_bottom,Q1_cap],1e3)

95.3
97.6
38.8


In [128]:
Ec(Q1_cap)

222.5

In [137]:
q_name='T2_'
T2_cap_ground_top=data[q_name+'large'][8]+data[q_name+'large'][9]
T2_cap_ground_bottom=data[q_name+'small'][9]+data[q_name+'small'][8]
T2_cap_top_bottom=-data[q_name+'small'][8]
T2_cap=cap_func(T2_cap_top_bottom,T2_cap_ground_bottom,T2_cap_ground_top)

In [140]:
cap_pretty_print([T2_cap,T2_cap_top_bottom,T2_cap_ground_bottom,T2_cap_ground_top],1e3)

72.0
30.3
61.2
130.8


In [141]:
Ec(T2_cap)

269.0

In [142]:
q_name='T3_'
T3_cap_ground_top=data[q_name+'large'][10]+data[q_name+'large'][11]
T3_cap_ground_bottom=data[q_name+'small'][11]+data[q_name+'small'][10]
T3_cap_top_bottom=-data[q_name+'small'][10]
T3_cap=cap_func(T3_cap_top_bottom,T3_cap_ground_bottom,T3_cap_ground_top)

In [144]:
cap_pretty_print([T3_cap,T3_cap_top_bottom,T3_cap_ground_bottom,T3_cap_ground_top],1e3)

69.3
20.7
57.6
310.5


In [145]:
Ec(T3_cap)

279.6

In [152]:
#Q1_T2 coupling
Ccq1t2=np.abs(data['T2_small'][2]-data['T2_small'][1])
Ccq1t2=np.abs(data['T2_small'][2]-data['T2_small'][1])

In [176]:
Ccq1t2,data['T2_large'][2],data['T2_large'][1]

(0.014317999999999999, -0.016342, -0.002024)

In [179]:
def coupling(Cc,Cq1,Cq2,w1,w2):
    g=(Cc/(np.sqrt(Cq1*Cq2)))*np.sqrt(w1*1e9*w2*1e9)/2
    return g/1e6# return g/2pi in MHz

In [181]:
wq1=4.5
wqt2=5.5
coupling(Ccq1t2/3,T2_cap,Q1_cap,wq1,wqt2)

149.96070780672497

In [165]:
Ccq1t2/np.sqrt(T2_cap*Q1_cap)

0.18085941857855511

In [166]:
np.sqrt(4.5*5.5)*0.18

0.8954886933959579

In [169]:
Ccq1t2*1e3,T2_cap*1e3,Q1_cap*1e3

(14.318, 72.00257018887187, 87.0431213332711)